# Import bibliotecas

In [ ]:
from tensorflow.keras.losses import binary_crossentropy
from sklearn.model_selection import train_test_split
from google.colab import files
import tensorflow as tf
from PIL import Image

import matplotlib.pyplot as plt
import seaborn as sns
import nibabel as nib
import numpy as np
import glob
import os

seed_val = 1
np.random.seed(seed_val)
tf.random.set_seed(seed_val)

# Lendo as imagens

Existem imagens de dimensões diversas, estou considerando apenas as que tem 320, 320

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_x = '/content/drive/MyDrive/prostate/imagesTr/*.nii'

X = []

for p in glob.iglob(path_x):
    nii_obj = nib.load(p).get_fdata()
    slices = np.array(nii_obj).shape[2]
    for i in range(slices):
        if(len(nii_obj[:, :, i, 0]) == 320):
            img = np.array(nii_obj[:, :, i, 0])
            mx = np.max(img)
            norm = img/mx
            norm = np.expand_dims(norm, axis=-1)
            norm = tf.convert_to_tensor(norm, dtype=tf.float32)
            X.append(norm)
        else:
            pass


X = np.array(X)

In [ ]:
path_labels = '/content/drive/MyDrive/prostate/labelsTr/*.nii'

labels = []
n_classes = 3

for p in glob.iglob(path_labels):
    nii_obj = nib.load(p).get_fdata()
    slices = np.array(nii_obj).shape[2]
    
    for i in range(slices):
        if(len(nii_obj[:, :, i]) == 320):
            img = np.array(nii_obj[:, :, i])
            img = np.expand_dims(img, axis=-1)
            img = tf.convert_to_tensor(img, dtype=tf.int32)

            aux = []

            for c in range(n_classes):
                mask = tf.equal(img[:,:,0], tf.constant(c))
                aux.append(tf.cast(mask, dtype=tf.int32))
            annotation = tf.stack(aux, axis=2)
            labels.append(annotation)
        else:
            pass

labels = np.array(labels)

In [ ]:
X_train, xtest, y_train, ytest = train_test_split(X, labels, train_size=0.6, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(xtest, ytest, train_size=0.5)

In [ ]:
X.shape

(540, 320, 320, 1)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

(324, 320, 320, 1)
(324, 320, 320, 3)
(108, 320, 320, 1)
(108, 320, 320, 3)
(108, 320, 320, 1)
(108, 320, 320, 3)


# Compilando rede U-net








In [ ]:
class Rede(tf.keras.models.Model):

    def __init__(self, n_filters, n_classes):
        super(Rede, self).__init__()
        self.n_filters = n_filters
        self.n_classes = n_classes

        self.max_pooling = tf.keras.layers.MaxPooling2D(pool_size=(2,2))

        self.encoder_block_0_conv_1 = tf.keras.layers.Conv2D(
            self.n_filters, (5,5), activation='relu',padding='same', kernel_initializer="he_normal")
        self.encoder_block_0_conv_2 = tf.keras.layers.Conv2D(
            self.n_filters, (5,5), activation='relu',padding='same', kernel_initializer="he_normal")
        self.encoder_block_0_BN = tf.keras.layers.BatchNormalization()

        self.encoder_block_1_conv_1 = tf.keras.layers.Conv2D(
            self.n_filters*2, (5,5), activation='relu',padding='same', kernel_initializer="he_normal")
        self.encoder_block_1_conv_2 = tf.keras.layers.Conv2D(
            self.n_filters*2, (5,5), activation='relu',padding='same', kernel_initializer="he_normal")
        self.encoder_block_1_BN = tf.keras.layers.BatchNormalization()

        self.encoder_block_2_conv_1 = tf.keras.layers.Conv2D(
            self.n_filters*4, (5,5), activation='relu',padding='same', kernel_initializer="he_normal")
        self.encoder_block_2_conv_2 = tf.keras.layers.Conv2D(
            self.n_filters*4, (5,5), activation='relu',padding='same', kernel_initializer="he_normal")
        self.encoder_block_2_BN = tf.keras.layers.BatchNormalization()

        self.encoder_block_3_conv_1 = tf.keras.layers.Conv2D(
            self.n_filters*8, (5,5), activation='relu',padding='same', kernel_initializer="he_normal")
        self.encoder_block_3_conv_2 = tf.keras.layers.Conv2D(
            self.n_filters*8, (5,5), activation='relu',padding='same', kernel_initializer="he_normal")
        self.encoder_block_3_BN = tf.keras.layers.BatchNormalization()

        self.encoder_block_4_conv_1 = tf.keras.layers.Conv2D(
            self.n_filters*16, (5,5), activation='relu',padding='same', kernel_initializer="he_normal")
        self.encoder_block_4_conv_2 = tf.keras.layers.Conv2D(
            self.n_filters*16, (5,5), activation='relu',padding='same', kernel_initializer="he_normal")
        self.encoder_block_4_BN = tf.keras.layers.BatchNormalization()

        ########################################################################

        self.decoder_block_0_convT = tf.keras.layers.Conv2DTranspose(
            self.n_filters*8, (2,2), strides=(2,2), padding='same', kernel_initializer="he_normal")
        self.decoder_block_0_conv_1 = tf.keras.layers.Conv2D(
            self.n_filters*8, (2,2), activation='relu',padding='same', kernel_initializer="he_normal")
        self.decoder_block_0_conv_2 = tf.keras.layers.Conv2D(
            self.n_filters*8, (2,2), activation='relu',padding='same', kernel_initializer="he_normal")

        self.decoder_block_1_convT = tf.keras.layers.Conv2DTranspose(
            self.n_filters*4, (2,2), strides=(2,2), padding='same', kernel_initializer="he_normal")
        self.decoder_block_1_conv_1 = tf.keras.layers.Conv2D(
            self.n_filters*4, (2,2), activation='relu',padding='same', kernel_initializer="he_normal")
        self.decoder_block_1_conv_2 = tf.keras.layers.Conv2D(
            self.n_filters*4, (2,2), activation='relu',padding='same', kernel_initializer="he_normal")

        self.decoder_block_2_convT = tf.keras.layers.Conv2DTranspose(
            self.n_filters*2, (2,2), strides=(2,2), padding='same', kernel_initializer="he_normal")
        self.decoder_block_2_conv_1 = tf.keras.layers.Conv2D(
            self.n_filters*2, (2,2), activation='relu',padding='same', kernel_initializer="he_normal")
        self.decoder_block_2_conv_2 = tf.keras.layers.Conv2D(
            self.n_filters*2, (2,2), activation='relu',padding='same', kernel_initializer="he_normal")

        self.decoder_block_3_convT = tf.keras.layers.Conv2DTranspose(
            self.n_filters, (2,2), strides=(2,2), padding='same', kernel_initializer="he_normal")
        self.decoder_block_3_conv_1 = tf.keras.layers.Conv2D(
            self.n_filters, (2,2), activation='relu',padding='same', kernel_initializer="he_normal")
        self.decoder_block_3_conv_2 = tf.keras.layers.Conv2D(
            self.n_filters, (2,2), activation='relu',padding='same', kernel_initializer="he_normal")

        self.decoder_final = tf.keras.layers.Conv2D(
            self.n_classes, (1,1), activation='softmax', padding='same')

        self.dropout = tf.keras.layers.Dropout(0.3)

    
    def call(self, inputs, training=True):

        out = self.encoder_block_0_conv_1(inputs)
        out = self.encoder_block_0_conv_2(out)
        # out = self.encoder_block_0_BN(out)
        out = self.max_pooling(out)
        skip_0 = out

        out = self.encoder_block_1_conv_1(out)
        out = self.encoder_block_1_conv_2(out)
        # out = self.encoder_block_1_BN(out)
        out = self.max_pooling(out)
        skip_1 = out

        out = self.encoder_block_2_conv_1(out)
        out = self.encoder_block_2_conv_2(out)
        # out = self.encoder_block_2_BN(out)
        out = self.max_pooling(out)
        skip_2 = out

        out = self.encoder_block_3_conv_1(out)
        out = self.encoder_block_3_conv_2(out)
        out = self.dropout(out, training=training)
        # out = self.encoder_block_3_BN(out)
        out = self.max_pooling(out)
        skip_3 = out

        out = self.encoder_block_4_conv_1(out)
        out = self.encoder_block_4_conv_2(out)
        out = self.dropout(out, training=training)
        # out = self.encoder_block_4_BN(out)

        ########################################################################
        out = tf.keras.layers.concatenate([out, skip_3])
        # out = tf.keras.layers.concatenate([out, skip_2])

        out = self.decoder_block_0_convT(out)
        out = tf.keras.layers.concatenate([out, skip_2])
        out = self.decoder_block_0_conv_1(out)
        out = self.decoder_block_0_conv_2(out)

        out = self.decoder_block_1_convT(out)
        out = tf.keras.layers.concatenate([out, skip_1])
        out = self.decoder_block_1_conv_1(out)
        out = self.decoder_block_1_conv_2(out)

        out = self.decoder_block_2_convT(out)
        out = tf.keras.layers.concatenate([out, skip_0])
        out = self.decoder_block_2_conv_1(out)
        out = self.decoder_block_2_conv_2(out)

        out = self.decoder_block_3_convT(out)
        out = self.decoder_block_3_conv_1(out)
        out = self.decoder_block_3_conv_2(out)

        out = self.decoder_final(out)

        return out

In [ ]:
model = None
model = Rede(32, 3)

In [ ]:
epocas = 30
lr = 0.001
otimizador = tf.keras.optimizers.Adam(learning_rate=lr)
loss_fn = tf.keras.losses.CategoricalCrossentropy()
metrics=[tf.keras.metrics.MeanIoU(num_classes=3)]


model.compile(optimizer=otimizador,
              loss=loss_fn,
              metrics=[metrics])

history = model.fit(X_train, 
                    y_train, 
                    epochs=epocas, 
                    batch_size=6, 
                    shuffle=True, 
                    validation_data=(X_val, y_val))

Epoch 1/30
54/54 [==============================] - 1023s 19s/step - loss: 0.8197 - mean_io_u_1: 0.3612 - val_loss: 0.1842 - val_mean_io_u_1: 0.3337
Epoch 2/30
54/54 [==============================] - 1022s 19s/step - loss: 0.1210 - mean_io_u_1: 0.5074 - val_loss: 0.1033 - val_mean_io_u_1: 0.4037
Epoch 3/30
54/54 [==============================] - 1018s 19s/step - loss: 0.0841 - mean_io_u_1: 0.6989 - val_loss: 0.0862 - val_mean_io_u_1: 0.4747
Epoch 4/30
54/54 [==============================] - 1016s 19s/step - loss: 0.0839 - mean_io_u_1: 0.6381 - val_loss: 0.0752 - val_mean_io_u_1: 0.5804
Epoch 5/30
54/54 [==============================] - 1015s 19s/step - loss: 0.0784 - mean_io_u_1: 0.7094 - val_loss: 0.0744 - val_mean_io_u_1: 0.7494
Epoch 6/30
17/54 [========>.....................] - ETA: 10:40 - loss: 0.0820 - mean_io_u_1: 0.7076

In [ ]:
def plot_loss(history, title):
    plt.plot(history.history['loss'], label='train-loss', color='purple')
    plt.plot(history.history['val_loss'], label='val-loss', color='blue')
    plt.title(f'Train and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'{title}.png')
    files.download(f'{title}.png')

exp = 'loss exp_4'
plot_loss(history, exp)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
model.save_weights('/content/')

In [ ]:
class Visualizer():
    def __init__(self, model, X_test, y_test, n_classes):
        self.model = model
        self.X_test = X_test
        self.y_test = y_test
        self.n_classes = n_classes
        self.colors = sns.color_palette(None, self.n_classes)

    def __give_color(self, y):
        seg_img = np.zeros((y.shape[0], y.shape[1], 3)).astype('float')
        for c in range(self.n_classes):
            segc = (y == c)
            print(self.colors[c])
            seg_img[:, :, 0] += segc * (self.colors[c][0] * 255.0)
            seg_img[:, :, 1] += segc * (self.colors[c][1] * 255.0)
            seg_img[:, :, 2] += segc * (self.colors[c][2] * 255.0)

        return seg_img

    def run(self, sample_idx):
        fig = plt.figure()
        X = self.X_test[sample_idx]
        y = self.__give_color(np.argmax(self.y_test[sample_idx], axis=2))
        y_pred = self.model.predict(X[np.newaxis, ...])
        y_pred = self.__give_color(np.argmax(y_pred[0, :, :, :], axis=2))

        f, ax = plt.subplots(nrows=1, ncols=3, figsize=(15,8))

        ax[0].imshow(X[:, :, 0], cmap='gray')
        ax[0].set_title('X')
        ax[0].axis('off')

        ax[1].imshow(y.astype(np.uint8))
        ax[1].set_title('Mask')
        ax[1].axis('off')

        ax[2].imshow(y_pred.astype(np.uint8))
        ax[2].set_title('Mask predicted')
        ax[2].axis('off')
        plt.tight_layout()
        plt.show()

In [ ]:
visualizer = Visualizer(model=model,
                        X_test=X_test,
                        y_test=y_test,
                        n_classes=3)
for x in range(100):
  if x%25==0:
    visualizer.run(sample_idx=x)

In [ ]:
unet3_1 = [
[15, 1],[15, 1],[2, 1],

[13, 1],[13, 1],[2, 1],

[11, 1],[11, 1],[2, 1],

[7, 1],[7, 1],[2, 1],

[5, 1],[5, 1],

[5, 2],[7, 1],[7, 1],

[5, 2],[11, 1],[11, 1],

[5, 2],[13, 1],[13, 1],

[5, 2],[15, 1],[15, 1],

[1, 1]]

unetA = [
[3, 1],[3, 1],[2, 1],

[3, 1],[3, 1],[2, 1],

[3, 1],[3, 1],[2, 1],

[3, 1],[3, 1],[2, 1],

[3, 1],[3, 1],

[3, 2],[3, 1],[3, 1],

[3, 2],[3, 1],[3, 1],

[3, 2],[3, 1],[3, 1],

[3, 2],[3, 1],[3, 1],

[1, 1]]

unetB = [
[5, 1],[5, 1],[2, 1],

[5, 1],[5, 1],[2, 1],

[5, 1],[5, 1],[2, 1],

[5, 1],[5, 1],

[5, 2],[5, 1],[5, 1],

[5, 2],[5, 1],[5, 1],

[5, 2],[5, 1],[5, 1],

[1, 1]]

unet3_2 = [
[3, 1],[3, 1],[2, 1],

[5, 1],[5, 1],[2, 1],

[7, 1],[7, 1],[2, 1],

[11, 1],[11, 1],[2, 1],

[13, 1],[13, 1],


[5, 2],[5, 1],[5, 1],

[5, 2],[7, 1],[7, 1],

[5, 2],[11, 1],[11, 1],

[5, 2],[13, 1],[13, 1],

[1, 1]]

unet3_3 = [
[7, 1],[7, 1],[2, 1],

[5, 1],[5, 1],[2, 1],

[3, 1],[3, 1],[2, 1],

[3, 1],[3, 1],[2, 1],

[3, 1],[3, 1],


[3, 2],[3, 1],[3, 1],

[5, 2],[5, 1],[5, 1],

[5, 2],[5, 1],[5, 1],

[3, 2],[7, 1],[7, 1],

[1, 1]]

unet3_4 = [
[5, 1],[5, 1],[2, 1],

[5, 1],[5, 1],[2, 1],

[3, 1],[3, 1],[2, 1],

[2, 1],[2, 1],[2, 1],

[2, 1],[2, 1],


[2, 2],[2, 1],[2, 1],

[3, 2],[3, 1],[3, 1],

[5, 2],[5, 1],[5, 1],

[5, 2],[5, 1],[5, 1],

[1, 1]]

unet3_5 = [
[25, 1],[25, 1],[2, 1],

[5, 1],[5, 1],[2, 1],

[3, 1],[3, 1],[2, 1],

[2, 1],[2, 1],[2, 1],

[2, 1],[2, 1],


[2, 2],[2, 1],[2, 1],

[3, 2],[3, 1],[3, 1],

[5, 2],[5, 1],[5, 1],

[5, 2],[5, 1],[5, 1],

[1, 1]]

unet3_6 = [
[5, 1],[5, 1],[2, 1],

[5, 1],[5, 1],[2, 1],

[3, 1],[3, 1],[2, 1],

[3, 1],[3, 1],[2, 1],

[3, 1],[3, 1],


[3, 2],[3, 1],[3, 1],

[3, 2],[3, 1],[3, 1],

[3, 2],[5, 1],[5, 1],

[3, 2],[5, 1],[5, 1],

[1, 1]]

unet3_7 = [
[2, 1],[2, 1],[2, 1],

[2, 1],[2, 1],[2, 1],

[2, 1],[2, 1],[2, 1],

[2, 1],[2, 1],[2, 1],

[2, 1],[2, 1],


[2, 2],[2, 1],[2, 1],

[2, 2],[2, 1],[2, 1],

[2, 2],[2, 1],[2, 1],

[2, 2],[2, 1],[2, 1],

[1, 1]]

unet4 = [
[5, 1],[5, 1],[2, 1],

[3, 1],[3, 1],[2, 1],

[3, 1],[3, 1],[2, 1],

[3, 1],[3, 1],[2, 1],

[3, 1],[3, 1],


[2, 2],[2, 1],[2, 1],

[2, 2],[2, 1],[2, 1],

[2, 2],[2, 1],[2, 1],

[2, 2],[2, 1],[2, 1],

[1, 1]] 

unet41 = [[5, 1],[5, 1],[2, 1],

[3, 1],[3, 1],[2, 1],

[3, 1],[3, 1],[2, 1],

[3, 1],[3, 1],[2, 1],

[3, 1],[3, 1]]

unet42 = [[2, 2],[2, 1],[2, 1],

[2, 2],[2, 1],[2, 1],

[2, 2],[2, 1],[2, 1],

[2, 2],[2, 1],[2, 1],

[1, 1]]

In [ ]:
def calcula_trf(dados):

    trf_ant = 0
    s = 1
    for dado in dados:
        k = dado[0]
        trf = trf_ant + ((k - 1) * s)
        # print(f'TRF: {trf_ant} + {(k - 1)} * {s} = {trf}')
        trf_ant = trf
        s = s * dado[1]
        
    return trf

print(f'A {calcula_trf(unetA)}')
print(f'B {calcula_trf(unetB)}')
print(f'3.1 {calcula_trf(unet3_1)}')
print(f'3.2 {calcula_trf(unet3_2)}')
print(f'3.3 {calcula_trf(unet3_3)}')
print(f'3.4 {calcula_trf(unet3_4)}')
print(f'3.5 {calcula_trf(unet3_5)}')
print(f'3.6 {calcula_trf(unet3_6)}')
print(f'3.7 {calcula_trf(unet3_7)}')
print(f'4 {calcula_trf(unet4)}')
print(f'4encoder {calcula_trf(unet41)}')
print(f'4decoder {calcula_trf(unet42)}')

A 174
B 175
3.1 900
3.2 740
3.3 374
3.4 293
3.5 333
3.6 278
3.7 89
4 103
4encoder 28
4decoder 75
